# Exploratory Data Analysis (EDA) for the Amazon Customer Reviews Dataset

In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from datasets import load_dataset, get_dataset_config_names

E:\Python Scripts\recsys\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

Using device: cuda


## Data Loading (Amazon Reviews Dataset 2023)
The dataset is loaded from the Hugging Face datasets library.

In [43]:
# Select 3 categories to highlight cross-domain transfer
TARGET_DOMAINS = ["Digital_Music", "Movies_and_TV", "Video_Games"]
MIN_USER_INTERACTIONS = 5
MIN_ITEM_INTERACTIONS = 5
POSITIVE_THRESHOLD = 4.0  # Ratings >= 4.0 are considered positive

# Load the dataset
def load_amazon_reviews(domains, max_per_domain=100000):
    dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023",
                           f"raw_review_{domains}",
                           trust_remote_code=True)

    rows = []
    for i, r in enumerate(dataset["full"]):
        if i >= max_per_domain:
            break
        rows.append({
            "user": r["user_id"],
            "item": r["parent_asin"],
            "rating": float(r["rating"]),
            "title": r.get("title", ""),
            "text": r.get("text", ""),
            "timestamp": int(r["timestamp"])
        })

    return pd.DataFrame(rows)

dfs = [load_amazon_reviews(dom, max_per_domain=100000) for dom in TARGET_DOMAINS]
df = pd.concat(dfs, ignore_index=True)

## Filtering dataset
* Remove low ratings: We consider ratings >= 4.0 as positive.
* Remove exact duplicates: Ensure that each user-item interaction is unique.
* Reduce item sparsity: Ensure that every remaining item has at least a certain number of interactions.
* Reduce user sparsity: Ensure that every user has at least a certain number of interactions.

In [44]:
# Remove low ratings, as we are building a model for positive recommendations (make implicit feedback)
def remove_low_ratings(dataframe, min_rating=4.0, keep_rating=True):
    dataframe["label"] = (dataframe["rating"] >= min_rating).astype(int)
    if not keep_rating:
        dataframe = dataframe.drop(columns=["rating"])
    return dataframe

df = remove_low_ratings(df, POSITIVE_THRESHOLD, keep_rating=True)
df.head()

,user,item,rating,title,text,timestamp,label
0,AFUOYIZBU3MTBOLYKOJE5Z35MBDA,B004RQ2IRG,5.0,Nice,If i had a dollar for how many times I have pl...,1618972613292,1
1,AHGAOIZVODNHYMNCBV4DECZH42UQ,B0026UZEI0,5.0,Excellent,awesome sound - cant wait to see them in perso...,1308167525000,1
2,AFGEM6BXCYHUILEOA3P2ZYBEF2TA,B0055JSYHC,5.0,Great service,This is a great cd. Good music and plays well....,1615838793006,1
3,AH3OG6QD6EDJGZRVCFKV4B66VWNQ,B000F9SMUQ,1.0,No good,"These are not real German singers, they have a...",1405219741000,0
4,AFW2PDT3AMT4X3PYQG7FJZH5FXFA,B0049D1WVK,3.0,"Cool concept, so-so execution...",I first heard this playing in a Nagoya shop an...,1309029595000,0


In [45]:
# Reduce item sparsity to ensure that every remaining item has at least certain number
# of interactions, and that every user has at least certain number of interactions
def filter_interactions(dataframe, min_user_interactions, min_item_interactions):
    # Removing exact duplicates first
    print(f"Dataframe before removing exact duplicates: {len(dataframe)}")
    dataframe = dataframe.drop_duplicates()
    print(f"Dataframe before interactions filtering: {len(dataframe)}")

    while True:
        before = len(dataframe)
        # Count interactions per user and item
        user_counts = dataframe.groupby("user")["item"].count()
        item_counts = dataframe.groupby("item")["user"].count()
        # Filter users and items based on interaction counts
        users_to_keep = set(user_counts[user_counts >= min_user_interactions].index)
        items_to_keep = set(item_counts[item_counts >= min_item_interactions].index)
        # Filter the DataFrame
        dataframe = dataframe[dataframe["user"].isin(users_to_keep) & dataframe["item"].isin(items_to_keep)]
        after = len(dataframe)
        if after == before:
            break
    print(f"Dataframe after interactions filtering: {after}")
    return dataframe

df = filter_interactions(df, MIN_USER_INTERACTIONS, MIN_ITEM_INTERACTIONS)

Dataframe before removing exact duplicates: 300000
Dataframe before interactions filtering: 298841
Dataframe after interactions filtering: 16514


,user,item,rating,title,text,timestamp
55560,AEJZE4VZLLYKJQD2EYTUHN76ITZQ,B000MNP2XU,4.0,I have previous cd and like his music so I am ...,I have previous cd and like his music so I am ...,1408310605000
55561,AEJZE4VZLLYKJQD2EYTUHN76ITZQ,B000MNP2XU,4.0,I have previous cd and like his music so I am ...,I have previous cd and like his music so I am ...,1408310605000
55562,AEJZE4VZLLYKJQD2EYTUHN76ITZQ,B000MNP2XU,4.0,I have previous cd and like his music so I am ...,I have previous cd and like his music so I am ...,1408310605000
55563,AEJZE4VZLLYKJQD2EYTUHN76ITZQ,B000MNP2XU,4.0,I have previous cd and like his music so I am ...,I have previous cd and like his music so I am ...,1408310605000
55564,AEJZE4VZLLYKJQD2EYTUHN76ITZQ,B000MNP2XU,4.0,I have previous cd and like his music so I am ...,I have previous cd and like his music so I am ...,1408310605000
55565,AEJZE4VZLLYKJQD2EYTUHN76ITZQ,B000MNP2XU,4.0,I have previous cd and like his music so I am ...,I have previous cd and like his music so I am ...,1408310605000
82158,AHPHH5CS7IJBCH2XI7FZGWHJDCIQ,B07ZVNNZPT,5.0,Late XMas Gift To Myself,Arrived on time. Love it!,1581912780074
82159,AHPHH5CS7IJBCH2XI7FZGWHJDCIQ,B015G0BLBA,3.0,I love the cover but it's not the Japanese ver...,I love the cover but it's not the Japanese ver...,1503969794679
82160,AHPHH5CS7IJBCH2XI7FZGWHJDCIQ,B07ZVNNZPT,5.0,Late XMas Gift To Myself,Arrived on time. Love it!,1581912780074
82161,AHPHH5CS7IJBCH2XI7FZGWHJDCIQ,B015G0BLBA,3.0,I love the cover but it's not the Japanese ver...,I love the cover but it's not the Japanese ver...,1503969794679
